<a href="https://colab.research.google.com/github/KonradWasikiewicz/DW_Matrix/blob/master/Men's_shoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
import pandas as pd 
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [6]:
cd "/content/drive/My Drive/Colab Notebooks/DW_Matrix"

/content/drive/My Drive/Colab Notebooks/DW_Matrix


In [7]:
ls data

men_shoes.csv  shoes_prices.csv


In [0]:
df = pd.read_csv("data/men_shoes.csv", low_memory=False)

In [0]:
def run_model(feats,model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring= 'neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [22]:
df['brand_cat']=df['brand'].map(lambda x:str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [23]:
model = RandomForestRegressor(max_depth=5, n_estimators=100,random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan':return output_dict

  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key]=value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [40]:
keys = set()
df['features_parsed'].map(lambda x:keys.update(x.keys()))

len(keys)

476

In [0]:
def get_name_feat(key):
  return 'feat_' + key 

for key in keys:
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan) 

In [43]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_fit:', 'feat_charge time (minutes)', 'feat_style:',
       'feat_vehicle type', 'feat_athlete', 'feat_is polarized',
       'feat_removable liner', 'feat_part type', 'feat_gps',
       'feat_sports team'],
      dtype='object', length=526)

In [0]:
keys_stat = {}
for key in keys:  
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0]/df.shape[0]*100

In [53]:
{k:v for k,v, in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat']=df['feat_brand'].factorize()[0]
df['feat_brand_cat']=df['feat_color'].factorize()[0]
df['feat_gender_cat']=df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat']=df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat']=df['feat_material'].factorize()[0]

df['feat_sport_cat']=df['feat_sport'].factorize()[0]
df['feat_style_cat']=df['feat_style'].factorize()[0]

for key in keys:  
  df[get_name_feat(key)+'_cat'] = df[get_name_feat(key)].factorize()[0]

In [57]:
df['brand'] =df['brand'].map(lambda x: str(x).lower())
df[ df.brand == df.feat_brand ].shape

(8846, 531)

In [59]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'],model)


(-57.35121545495055, 4.20337257197596)

In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]


In [0]:
feats = ['brand_cat','feat_metal type_cat','feat_shape_cat','feat_brand_cat','feat_gender_cat','feat_material_cat', 'feat_style_cat', 'feat_sport_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats,model)

In [100]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats)

(-57.2695208246702, 4.177588244893792)


Weight,Feature
0.2557 ± 0.0080,brand_cat
0.1032 ± 0.0075,feat_material_cat
0.0252 ± 0.0026,feat_gender_cat
0.0175 ± 0.0015,feat_brand_cat
0.0127 ± 0.0011,feat_shape_cat
0.0091 ± 0.0015,feat_metal type_cat
0.0033 ± 0.0012,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [83]:
df[df['brand'] =='nike'].features_parsed.sample(5).values

array([{'sport': 'soccer', 'heel height': 'flat (0 to 1/2")', 'gender': 'men', 'shade': 'laser orange total orange persian violet black', 'color': 'orange', 'name': 'tiempo legend v', 'features & fastening': 'laces'},
       {'season': 'all-season', 'material': 'leather', 'gender': 'men', 'shoe size': '7.5', 'color': 'brown', 'manufacturer part number': '555317 240', 'brand': 'nike', 'age group': 'adult', 'size': '6.5', 'model': '555317 240'},
       {'type': 'cleats', 'condition': 'new with box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'},
       {'style': 'track & sweat pants', 'pattern': 'solid', 'material': 'cotton blend', 'sleeve length': 'long sleeve', 'condition': 'new with tags'}],
      dtype=object)